In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pickle
import numpy as np
import cv2
import glob
from pathlib import Path
import matplotlib.pyplot as plt

from moviepy.editor import VideoFileClip
from IPython.display import HTML

import utils

In [11]:
from scipy.ndimage.measurements import label

with open("./output_data/training_info.p", 'rb') as f:
    train_info = pickle.load(f)

svc = train_info["model"]
X_scaler = train_info["scaler"]
cspace = train_info["cspace"]
orient = train_info["orient"]
pix_per_cell = train_info["pix_per_cell"]
cell_per_block = train_info["cell_per_block"]
hog_channel = train_info["hog_channel"]
spatial_size = train_info["spatial_size"]
hist_bins = train_info["hist_bins"]


multi_scales = [
    (400, 500, 1.0),
    (400, 550, 1.2),
    (400, 600, 1.5),
    (450, 650, 1.7),
    (450, 700, 2.0),
]
threshold = 1
    
def process_image(image):
    """
    """
    multiscale_bboxes = []
    img = utils.image_preprocess(image)
    for search_info in multi_scales:
        ## get bbox
        ystart, ystop, scale = search_info
        bboxes = utils.find_cars(img, ystart, ystop, scale, svc, X_scaler, cspace, orient, pix_per_cell, cell_per_block, hog_channel, spatial_size, hist_bins)
        multiscale_bboxes = multiscale_bboxes + bboxes
    ## draw bboxes
    img_bbox = utils.draw_boxes(img, multiscale_bboxes)

    ## make heatmatp
    img_heat = np.zeros_like(img[:,:,0]).astype(np.float)
    img_heat = utils.add_heat(img_heat, multiscale_bboxes)

    ## apply thresh
    img_thresh = utils.apply_threshold(np.copy(img_heat), threshold)
    labels = label(img_thresh)
    img_label, nlabel = labels

    ## draw labeled bbox
    img_label_bbox = utils.draw_labeled_bboxes(img, labels)
            
    return img_label_bbox

In [12]:
white_output = 'output_video/sol_project_video.mp4'

##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
#clip = VideoFileClip("project_video.mp4").subclip(30, 45)
clip = VideoFileClip("project_video.mp4")
print(clip.fps)
clip = clip.set_fps(5)
white_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

25.0
[MoviePy] >>>> Building video output_video/sol_project_video.mp4
[MoviePy] Writing video output_video/sol_project_video.mp4



100%|█████████▉| 252/253 [04:27<00:01,  1.06s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video/sol_project_video.mp4 

CPU times: user 13min 18s, sys: 17.3 s, total: 13min 35s
Wall time: 4min 28s
